In [ ]:
import pandas as pd
import requests
from csv import reader
import os


port = 6006
http://localhost:6007/experiment/defaultExperimentId/data/plugin/scalars/scalars?tag=predictaion_1&run=predictaion_1_label&format=csv

def URLs(fold, trial):
    URLs_dict = {
        "train_accuracy": f"http://localhost:{port}/data/plugin/scalars/scalars?tag=epoch_accuracy&run=fold{fold}%5C{trial}%5Cexecution0%5Ctrain&format=csv",
        "val_accuracy": f"http://localhost:{port}/data/plugin/scalars/scalars?tag=epoch_accuracy&run=fold{fold}%5C{trial}%5Cexecution0%5Cvalidation&format=csv",
        "val_loss": f"http://localhost:{port}/data/plugin/scalars/scalars?tag=epoch_loss&run=fold{fold}%5C{trial}%5Cexecution0%5Cvalidation&format=csv",
        "train_loss": f"http://localhost:{port}/data/plugin/scalars/scalars?tag=epoch_loss&run=fold{fold}%5C{trial}%5Cexecution0%5Ctrain&format=csv",
    }
    return URLs_dict


def tb_data(log_dir, mode, fold, num_trials):
    trials = os.listdir(log_dir)
    fdf = {}
    for i, trial in enumerate(trials[:num_trials]):
        r = requests.get(URLs(fold, trial)[mode], allow_redirects=True)
        data = r.text
        data_csv = reader(data.splitlines())
        data_csv = list(data_csv)
        df = pd.DataFrame(data_csv)
        headers = df.iloc[0]
        df = pd.DataFrame(df.values[1:], columns=headers)
        if i == 0:
            fdf["Step"] = df["Step"]
        fdf[f"trial {trial}"] = df["Value"]
    fdf = pd.DataFrame(fdf)
    return fdf
